In [2]:
# import libraries

import cv2
import os
import json
import numpy as np

In [3]:
# main paths

images_dir = './samples'
input = 'input.json'
output = 'output.json'

In [4]:
# check if a box is inside another

def calculate_intersection_area(square1, square2):
    x1, y1, w1, h1 = square1
    x2, y2, w2, h2 = square2
    
    x_intersect = max(x1, x2)
    y_intersect = max(y1, y2)
    w_intersect = min(x1 + w1, x2 + w2) - x_intersect
    h_intersect = min(y1 + h1, y2 + h2) - y_intersect
    
    if w_intersect <= 0 or h_intersect <= 0:
        return 0
    
    intersection_area = w_intersect * h_intersect
    return intersection_area

def is_intersected(square1, square2,ratio_threshold):
    area_square1 = square1[2] * square1[3]  # area = width * height
    area_square2 = square2[2] * square2[3]
    area_intersection = calculate_intersection_area(square1, square2)
    
    ratio1 = area_intersection / area_square1
    ratio2 = area_intersection / area_square2
    
    if ratio1 > ratio_threshold and ratio1 > ratio2:
        return True
    else:
        return False

In [5]:
# open an image

with open(input, 'r') as json_file:
    data = json.load(json_file)

image_paths = data['image_files']
image_index = 0

original_image = cv2.imread(os.path.join(images_dir, image_paths[image_index]))
original_image = cv2.resize(original_image, (0, 0), fx = 0.15, fy = 0.15)

cv2.imshow('Original', original_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
# gray and hsv versions

gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

hsv_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2HSV)

cv2.imshow('Gray', gray_image)
cv2.imshow('HSV', hsv_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
def get_image_complexity(gray):
    sift = cv2.SIFT_create()
    keypoints, _ = sift.detectAndCompute(gray, None)
    num_keypoints = len(keypoints)
    threshold_value = 300
    if num_keypoints < threshold_value:
        complexity = "Simple"
    else:
        complexity = "Complex"
    return complexity

def median_background(hsv):
    hsv_flattened = hsv.reshape((-1, 3))
    hsv_flattened = hsv_flattened[hsv_flattened[:, 2] > 0]
    background_color = np.median(hsv_flattened, axis=0)
    return background_color

def limits(background_color, hue_tolerance, saturation_tolerance, value_tolerance):
    lower_limit_0 = max(background_color[0] - hue_tolerance, 0)
    upper_limit_0 = min(background_color[0] + hue_tolerance, 180)
    lower_limit_1 = max(background_color[1] - saturation_tolerance, 0)
    upper_limit_1 = min(background_color[1] + saturation_tolerance, 255)
    lower_limit_2 = max(background_color[2] - value_tolerance, 0)
    upper_limit_2 = min(background_color[2] + value_tolerance, 255)
    lower_background = np.array([lower_limit_0, lower_limit_1, lower_limit_2])
    upper_background = np.array([upper_limit_0, upper_limit_1, upper_limit_2])
    return lower_background, upper_background

def background_median(hsv):
    background_color = median_background(hsv)
    hue_tolerance = 20
    saturation_tolerance = 90
    value_tolerance = 90
    lower_background, upper_background = limits(background_color, hue_tolerance, saturation_tolerance, value_tolerance)
    blur = cv2.blur(hsv, (5, 5))
    gray_mask = cv2.inRange(blur, lower_background, upper_background)
    non_gray_mask = cv2.bitwise_not(gray_mask)
    non_gray_mask = cv2.erode(non_gray_mask, None, iterations=3)
    return non_gray_mask

def kmeans_segments(image):
    reshaped_image = image.reshape((-1,1))
    reshaped_image = np.float32(reshaped_image)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    k = 50
    _, label, center = cv2.kmeans(reshaped_image, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    center = np.uint8(center)
    result = center[label.flatten()]
    result = result.reshape((image.shape))
    return result

def canny_edges(gray):
    gaussian = cv2.GaussianBlur(gray, (7, 7), 0)
    #kmeans = kmeans_segments(gaussian)
    canny = cv2.Canny(gaussian, 100, 200)
    return canny

def grab_cut(image):
    gaussian = cv2.GaussianBlur(image, (5, 5), 0)
    mask = np.zeros(gaussian.shape[:2], np.uint8)
    size = 50
    bb = (size, size, gaussian.shape[1] - size, gaussian.shape[0] - size)
    bgModel = np.zeros((1, 65), np.float64)
    fgModel = np.zeros((1, 65), np.float64)
    (mask, bgModel, fgModel) = cv2.grabCut(gaussian, mask, bb, bgModel, fgModel, 5, cv2.GC_INIT_WITH_RECT)
    output_mask = np.where((mask == cv2.GC_BGD) | (mask == cv2.GC_PR_BGD), 0, 1)
    output_mask = (output_mask * 255).astype("uint8")
    return output_mask

In [11]:
image = original_image.copy()
gray = gray_image.copy()
hsv = hsv_image.copy()

grabcut_mask = grab_cut(image)
grabcut_result = cv2.bitwise_and(image, image, mask=grabcut_mask)

# apply other masks along with grabcut (TO-DO)
bg_median_mask = background_median(hsv)
combined_mask = cv2.bitwise_and(grabcut_mask, bg_median_mask)
result = cv2.bitwise_and(image, image, mask=combined_mask)

contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True)

ratio_threshold = 0.5
removed = []
for i, contour in enumerate(contours):
    x, y, w, h = cv2.boundingRect(contour)
    inside_other_box = False
    for j in range(i+1, len(contours)):
        prev_x, prev_y, prev_w, prev_h = cv2.boundingRect(contours[j])
        if is_intersected((x, y, w, h), (prev_x, prev_y, prev_w, prev_h), ratio_threshold):
            inside_other_box = True
            break
        elif is_intersected((prev_x, prev_y, prev_w, prev_h),(x,y,w,h), ratio_threshold):
            removed.append(j)
    if inside_other_box or w * h < 400:
        removed.append(i)

contours = [contour for i, contour in enumerate(contours) if i not in removed]
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 255, 0), 2)

num_lego_pieces = len(contours)
print("Number of LEGO pieces detected:", num_lego_pieces)

cv2.imshow('Image', image)
cv2.imshow('Mask', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

Number of LEGO pieces detected: 1
